In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cassiopeia as cass
import matplotlib.pyplot as plt
import time
import datetime

from collections import Counter
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from csv import writer


In [23]:
column_names = ['ID','Rank', 'B_Top','B_Top_Kills', 'B_Jg','B_Jg_Kills', 'B_Mid','B_Mid_Kills', 'B_Bot','B_Bot_Kills','B_Sup','B_Sup_Kills',
                'R_Top','R_Top_Kills', 'R_Jg','R_Jg_Kills', 'R_Mid','R_Mid_Kills', 'R_Bot','R_Bot_Kills','R_Sup','R_Sup_Kills','B_Win','R_Win']

df = pd.DataFrame()

In [24]:
df

""


In [25]:
cass.set_riot_api_key("RGAPI-facd70cc-4cd9-432f-92ee-6c4210435e51")
player = cass.Summoner(name="Artezzaa", region="NA")

In [28]:
def find_next_player(history):
    game = 0
    New_Player_Attempts = 0
    while True:
        New_Player_Attempts += 1
        match = history[game]
        randNum = np.random.randint(11)
        player = match.participants[randNum]
        if player.summoner.name in already_checked:
            if New_Player_Attempts == 10:
                game += 1
                New_Player_Attempts = 0
                continue
            else:
                continue
        break
    print(player.summoner.name)
    already_checked.append(player.summoner.name)
    return player.summoner.puuid

In [29]:
def add_match_history(puuid, games):
    global df
    match_history = cass.MatchHistory(puuid=puuid,continent="AMERICAS", region='NA',begin_index=0,end_index=games,queue=cass.Queue.ranked_solo_fives)
    for match in match_history:            
        try:
            if str(match.id) in df["ID"].values or str(match.id)[4:] in df["ID"].values:
                continue
        except:
            pass
        
        if match.duration < datetime.timedelta(hours=0, minutes=15, seconds=0):
            continue
        
        stat_import = pd.DataFrame(index=[0])
        
        p1 = match.participants[0]
        p2 = match.participants[1]
        p3 = match.participants[2]
        p4 = match.participants[3]
        p5 = match.participants[4]
        p6 = match.participants[5]
        p7 = match.participants[6]
        p8 = match.participants[7]
        p9 = match.participants[8]
        p10 = match.participants[9]
        
        player_list = [p1,p2,p3,p4,p5,p6,p7,p8,p9,p10]
        
        if type(match.id) == str:
            stat_import['ID'] = match.id[4:]
        else:
            stat_import['ID'] = str(match.id)
        stat_import['B_Win'] = match.blue_team.win
        stat_import['R_Win'] = match.red_team.win

        try:
            summoner_ID = p1.summoner.puuid
            test_player = cass.Summoner(puuid = summoner_ID, region="NA")
            entries = test_player.league_entries
            for entry in entries:
                if entry.queue == cass.Queue.ranked_solo_fives:
                    stat_import['Rank'] = entry.league.tier.name
        except:
            pass 
        
        try:     
            for player in player_list:
                
                if player.side.value == 100:
                    if player.individual_position.name == 'top_lane':

                        stat_import['B_Top'] = player.champion.name
                        stat_import['B_Top_Kills'] = player.stats.kda
                        
                    elif player.individual_position.name == 'jungle':

                        stat_import['B_Jg'] = player.champion.name
                        stat_import['B_Jg_Kills'] = player.stats.kda

                    elif player.individual_position.name == 'mid_lane':

                        stat_import['B_Mid'] = player.champion.name
                        stat_import['B_Mid_Kills'] = player.stats.kda

                    elif player.individual_position.name == 'bot_lane':

                        stat_import['B_Bot'] = player.champion.name
                        stat_import['B_Bot_Kills'] = player.stats.kda

                    elif player.individual_position.name == 'utility':

                        stat_import['B_Sup'] = player.champion.name
                        stat_import['B_Sup_Kills'] = player.stats.kda

                if player.side.value == 200:
                    if player.individual_position.name == 'top_lane':

                        stat_import['R_Top'] = player.champion.name
                        stat_import['R_Top_Kills'] = player.stats.kda

                    elif player.individual_position.name == 'jungle':

                        stat_import['R_Jg'] = player.champion.name
                        stat_import['R_Jg_Kills'] = player.stats.kda

                    elif player.individual_position.name == 'mid_lane':

                        stat_import['R_Mid'] = player.champion.name
                        stat_import['R_Mid_Kills'] = player.stats.kda

                    elif player.individual_position.name == 'bot_lane':

                        stat_import['R_Bot'] = player.champion.name
                        stat_import['R_Bot_Kills'] = player.stats.kda

                    elif player.individual_position.name == 'utility':

                        stat_import['R_Sup'] = player.champion.name
                        stat_import['R_Sup_Kills'] = player.stats.kda
        except:
           pass
        
        df = pd.concat([df,stat_import], ignore_index=True)

        with open("matches2.csv", "a") as f:
            stat_import.to_csv(path_or_buf=f,mode='a',index=False,header=False)

    return find_next_player(match_history)
    

In [30]:
i = 0
column_names = ['ID','Rank', 'B_Top','B_Top_Kills', 'B_Jg','B_Jg_Kills', 'B_Mid','B_Mid_Kills', 'B_Bot','B_Bot_Kills','B_Sup','B_Sup_Kills','R_Top','R_Top_Kills', 'R_Jg','R_Jg_Kills', 'R_Mid','R_Mid_Kills', 'R_Bot','R_Bot_Kills','R_Sup','R_Sup_Kills','B_Win','R_Win']
with open("matches2.csv", "w") as f:
    writer_obj = writer(f)
    writer_obj.writerow(column_names)
next_puuid = add_match_history(player.puuid, 10)
while i != 2:
    next_puuid = add_match_history(next_puuid, 10)
    i += 1

df.to_csv("matches3.csv",index=False,mode='w')

Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/c_dgoK4Dv3ZHJjkyvD1i8Wv_py3AcLgY_x7trjSgWeyms-aV1AtEONdMKJ1eP79ZAvFSu5Dn3_elsA/ids?start=0&count=10&queue=420
CapTJuiCyFruiTy
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/pE6IkgfXb3tHjhDlpnu0mEyI2wR0WgG2Ae4CYAFp6oS7v9b_mSIqci35dA2W_Q6JUjFzRDxWrhAOlg/ids?start=0&count=10&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4255343806
Making call: https://na1.api.riotgames.com/lol/league/v4/entries/by-summoner/9J152fLhDulsBZIekT4ffXMHysTczEhB2FtheYNTBSsSYBY
Making call: https://na1.api.riotgames.com/lol/league/v4/leagues/6e2e5841-1ae0-485f-9d8f-11cc2e7134b4
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4255287712
Making call: https://na1.api.riotgames.com/lol/league/v4/entries/by-summoner/HVMqHaJkwg6YyJWaqTji4TyG91NCM9TEOLXitvOBPEsjMCI
Making call: https://na1.api.riotgames.com/lol/league/v4/leagues/b74ef767-569a-46c7-8c5b

In [31]:
df

,ID,B_Win,R_Win,Rank,B_Top,B_Top_Kills,B_Jg,B_Jg_Kills,B_Mid,B_Mid_Kills,...,R_Top,R_Top_Kills,R_Jg,R_Jg_Kills,R_Mid,R_Mid_Kills,R_Bot,R_Bot_Kills,R_Sup,R_Sup_Kills
0,4254552847,False,True,diamond,Jayce,1.200000,Ekko,1.230769,Veigar,4.285714,...,Akali,3.428571,Talon,2.875000,Gragas,1.857143,Jhin,2.400000,Lux,2.300000
1,4251572080,True,False,diamond,Aatrox,1.500000,Jarvan IV,21.000000,Akali,1.800000,...,Mordekaiser,2.500000,Viego,1.000000,Karma,1.666667,Ezreal,0.875000,Leona,1.000000
2,4251403578,False,True,diamond,Camille,1.375000,Master Yi,1.142857,Yasuo,0.666667,...,Irelia,1.750000,Rammus,8.000000,Katarina,10.500000,Kai'Sa,8.500000,Leona,10.000000
3,4250631950,False,True,diamond,Akali,2.500000,Kayn,1.545455,Cassiopeia,2.571429,...,Fiora,1.888889,Jarvan IV,2.333333,Yasuo,2.888889,Kai'Sa,4.166667,Yuumi,5.200000
4,4249706191,True,False,diamond,Singed,14.000000,Talon,4.000000,Ahri,3.666667,...,Akali,0.600000,Graves,4.666667,Lissandra,1.500000,Jhin,0.857143,Nautilus,0.400000
5,4249710213,True,False,diamond,Akali,4.000000,Kha'Zix,13.000000,Cassiopeia,4.500000,...,Gangplank,2.333333,Olaf,0.625000,Ryze,0.714286,Aphelios,1.000000,Seraphine,1.000000
6,4247789680,False,True,platinum,Malphite,1.000000,Lee Sin,1.000000,Qiyana,0.545455,...,Akali,1.333333,Rek'Sai,12.000000,Rumble,6.333333,Jhin,10.000000,Morgana,2.500000
7,4247794168,True,False,diamond,Akali,10.500000,Vi,8.666667,Sylas,4.333333,...,Fiora,0.900000,Volibear,1.125000,Yasuo,0.769231,Caitlyn,2.000000,Taliyah,1.800000
8,4247736076,True,False,diamond,Irelia,3.000000,Fiddlesticks,6.333333,Akali,2.000000,...,Zeri,1.000000,Rek'Sai,1.857143,Karma,1.800000,Jinx,0.818182,Leona,2.750000
9,4246948912,False,True,diamond,Yorick,0.800000,Skarner,1.000000,Akali,0.625000,...,Camille,4.333333,Hecarim,14.000000,Ahri,5.000000,Kai'Sa,11.000000,Nautilus,3.000000


In [32]:
# https://github.com/meraki-analytics/cassiopeia/tree/master/examples